## Convert BioCxml to Pubannotation Json
## April 27

In [22]:

import pandas as pd

df = pd.read_csv ('../corona/manuscript/Supplemental_file6.csv')
print (df)

         pmid        type   id       entity_type  \
0    31991541  annotation   A2       Virus_other   
1    31991541  annotation   A3  Virus_SARS-CoV-2   
2    31991541  annotation   A4  Virus_SARS-CoV-2   
3    31991541  annotation   A5      Virus_family   
4    31991541  annotation   A1  Virus_SARS-CoV-2   
..        ...         ...  ...               ...   
150  31996494  annotation  A60  Virus_SARS-CoV-2   
151  31996494  annotation  A61      Virus_family   
152  31996494  annotation  A62      Virus_family   
153  31996494  annotation  A63  Virus_SARS-CoV-2   
154  31996494  annotation  A64  Virus_SARS-CoV-2   

                                          text location/reference  \
0                                     SARS-CoV              114:8   
1    most recent emergent group 2B coronavirus             293:41   
2                                    2019-nCoV               27:9   
3                                  Coronavirus              14:11   
4                            n

In [3]:
import collections
import csv
import sys
import codecs
from xml.dom.minidom import parse
import xml.dom.minidom
from bs4 import BeautifulSoup


with codecs.open("../corona/manuscript/Supplemental_file4.xml", "r", "utf-8") as file:##gold standard
    soup = BeautifulSoup(file, "html.parser")
#print(soup.prettify())

Since, Cord19 papers are identified by "sha" and "cord_id", I extracted those columns from "supplementary_file" for gold papers.
One of the papers did not exist at all in cord19 papers and one doesnot have neither "pmcid" nor "sha".

In [12]:
pmc_id = []
pub_id = []
cord_id = []
sha = []

with open("../files/gold_standard_corpus/gold_corpus_ids.txt",'r') as f:
    first_line = f.readline()
    for line in f:
        #print(line)
        pmc_id.append(line.strip().split(',')[2].replace('.xml.json',''))
        pub_id.append(str(int(float(line.strip().split(',')[3]))))
        cord_id.append(line.strip().split(',')[0])
        sha.append(line.strip().split(',')[1].split(';')[0])
        
print("pm_cid = {}".format(pmc_id))
print("pub_id ={} ".format(pub_id))
print("cord_id = {} ".format(cord_id))
print("sha = {} ".format(sha))

pm_cid = ['nan', 'PMC7003341', 'PMC7033720', 'PMC7159299', 'PMC7110798', 'PMC7094943', 'PMC6988272', 'PMC7054940', 'PMC7077245']
pub_id =['32013309', '32029004', '32020836', '31986264', '32007643', '32015508', '31992388', '32036774', '31991541'] 
cord_id = ['0lsniwyv', 'hmy8fs3g', 'ea99j2qn', '0nhgxoim', 'drqnrwdl', 'qele28zk', '0rxtati9', '1qkwsh6a', 't8s4s0wo'] 
sha = ['nan', 'fd28e6d03eef27b0454f13ca539dc1498242a4c2', '52dfa9fbd6dcf7c51fac5cb85a88fb154506a722', 'ea646b114ea01f978d205ed97405e1f2afed2dfd', '637939a0f462b9e821ff62fc20e9fedfe78df73e', '5734e3b81e16fe1976a129c5a0872716f3dd50b8', '28223ad437aa22ac2285bd9dd775e1415a69411a', '7bc69471836fbf9cf28e57926d95fc48e6f45964', '253cfd411f93ef88f702accd0fa195f24d1d2925'] 


I tried to extract title and abstract of the gold papers from kaggle database 

In [15]:
import json
for file in sha:
    if file!='nan':
        data = json.load(open("../Kaggle_data/gold_corpus_json/"+file+'.json'))
        jtopy=json.dumps(data,indent=4) #json.dumps take a dictionary as input and returns a string as output.
        dict_json=json.loads(jtopy) # json.loads take a string as input and returns a dictionary as output.
        index_sh = sha.index(dict_json["paper_id"])
        if 'abstract'in dict_json.keys():
            print("paper_id:",pmc_id[index_sh])
            #print("title:",dict_json["metadata"]["title"])
            #print("abstract: ", dict_json["abstract"])

paper_id: PMC7003341
paper_id: PMC7033720
paper_id: PMC7159299
paper_id: PMC7110798
paper_id: PMC7094943
paper_id: PMC6988272
paper_id: PMC7054940
paper_id: PMC7077245


In [18]:
from xml.etree import ElementTree

with open('../corona/manuscript/Supplemental_file4.xml', 'rt') as f:
    tree = ElementTree.parse(f)

source_id = []
source_db = []
text = []
denotation_array = []
annotation_id = []
begin = []
end = []


for node in tree.iter():

    if node.tag == "id":
        source_id.append(node.text)
        source_id.append(node.text)
    if node.tag =="infon" and node.attrib['key']=='BQ_FROM':
        source_db.append(node.text)
        source_db.append(node.text)
    if node.tag == 'annotation':
        annotation_id.append(node.attrib.get('id'))
    if node.tag=='location':
        span_begin = int(node.attrib.get('offset'))
        span_end   = int(span_begin) + int(node.attrib.get('length'))
        begin.append(span_begin)
        end.append(span_end)

###############################################################################
#print(text)
cord_uid = []
pmc_source_id = source_id
for pb in source_id:
    if pb in pub_id:
        index_s =  source_id.index(pb)
        index_p =  pub_id.index(pb)
        cord_uid.append(cord_id[index_p])
        if pmc_id[index_p] !='nan':
            pmc_source_id[index_s] = pmc_id[index_p]
            source_db[index_s] = "PMC"
        
    else:
        cord_uid.append(" ")
        
print(source_id)
print(source_db)
print(cord_uid)
        
#############################################################################
##This will write two json objects of title and abstract in one json file in Pubannotation format 

print('done!')    

ann_id_ctr = 0 
general_ctr = 0
documents = soup.find_all('document')
    #print(documents[1])
for i in documents:
    passages = i.find_all('passage')
    for k in passages:
        offset = int(k.find('offset').text)
        passage_text = k.find('text').text
        #print(passage_text)
        objects = k.find_all('annotation')
        denotation_array = []
        for obj in objects:
            obj_id = annotation_id[ann_id_ctr]
            info = obj.find_all('infon')
            for ann_info in info:
                if "key=\"type\"" in str(ann_info):
                    obj_description = str(ann_info.text)#+':'
                   # if "NCBI:txid" in str(ann_info):
                   #    obj_description = obj_description + str(ann_info.text)
            span_begin = begin[ann_id_ctr]-offset
            span_end   = end[ann_id_ctr]-offset
            ann_id_ctr = ann_id_ctr+1
            denotation_array.append({"id": obj_id, "span": {"begin": span_begin, "end": span_end}, "obj": obj_description})                                
        if general_ctr<20:
            with open('../files/gold_standard_corpus/'+source_id[general_ctr]+".json",'a') as out_file:
                #print(passage_text)
                tmp_dict = {
                    "cord_uid": cord_uid[general_ctr],
                    "sourcedb": source_db[general_ctr],
                    "sourceid": pmc_source_id[general_ctr],
                    "text": passage_text, # either abstract or title
                    "denotations": denotation_array  # all annotations
                    }
                out_file.write(json.dumps(tmp_dict,indent=4))
                out_file.close()
                general_ctr = general_ctr + 1
                #print(general_ctr)
                
        #
                                         
        


['PMC7077245', 'PMC7077245', '32013309', '32013309', 'PMC7110798', 'PMC7110798', 'PMC7054940', 'PMC7054940', 'PMC6988272', 'PMC6988272', 'PMC7033720', 'PMC7033720', 'PMC7094943', 'PMC7094943', 'PMC7159299', 'PMC7159299', 'PMC7003341', 'PMC7003341', '31996494', '31996494']
['PMC', 'PMC', 'PubMed', 'PubMed', 'PMC', 'PMC', 'PMC', 'PMC', 'PMC', 'PMC', 'PMC', 'PMC', 'PMC', 'PMC', 'PMC', 'PMC', 'PMC', 'PMC', 'PubMed', 'PubMed']
['t8s4s0wo', 't8s4s0wo', '0lsniwyv', '0lsniwyv', 'drqnrwdl', 'drqnrwdl', '1qkwsh6a', '1qkwsh6a', '0rxtati9', '0rxtati9', 'ea99j2qn', 'ea99j2qn', 'qele28zk', 'qele28zk', '0nhgxoim', '0nhgxoim', 'hmy8fs3g', 'hmy8fs3g', ' ', ' ']
done!


In [22]:

with codecs.open("../files/Updated_Gold_std.json", "r", "utf-8") as file:
    soupjson = BeautifulSoup(file, "html.parser")
#print(soupjson.prettify())

In [25]:
import json

JsonList = []
#print("Started Reading JSON file which contains multiple JSON document")
with open('../files/Updated_Gold_std.json') as json_file:
       for j in json_file:
            x = json.loads(j)
            JsonList.append(x)




## Convert gold standard papers to Cord19 format
### 29 of April

In [75]:
from xml.etree import ElementTree
JsonList = []
with open('Updated_gold_standard.json', 'rt') as j_f:
    for j in j_f:
        x = json.loads(j)
        JsonList.append(x)
        
    for j_ctr in range(len(JsonList)):
        if not j_ctr%2:
            paper_id = JsonList[j_ctr]["sourceid"]
            paper_title = JsonList[j_ctr]["text"]
        else:
            paper_abstract = JsonList[j_ctr]["text"]
            with open ("../Kaggle_data/gold_corpus_json/gold_papers/"+paper_id+".json",'w') as f:
                temp_dict = {
                    "paper_id" : paper_id,
                    "metadata" : {
                        "title":paper_title,
                        "authors":[]
                    },
                    "abstract":[
                       { 
                        "text":paper_abstract,
                        "cite_spans": [],
                        "ref_spans": [],
                        "section": "Abstract"

                       }
                    ],
                    "body_text": []
                     
                }
                
                f.write(json.dumps(temp_dict,indent=4))
                f.close()
            


